# Step 4

## Quantify WM-task performance with signal detection theory methods

We take in-scanner behavioral data and extract the exact number of specific types of responses to calculate discriminability and bias (d' and c').

In [37]:
import glob
import os

import numpy as np
import pandas as pd
import h5py as h5
import pystan

In [15]:
#f = h5.File('../data/reconstructed_WM.hdf5','r')
f = h5.File('/Users/marcel/projects/HCP/volumes_embedded_full.hdf5')
subjects = f['Working_memory/Subjects']
datapath = '~/HCP/data'

## Extract the relevant information from logs

### 2-back

In [16]:
d = np.zeros([len(subjects), 13])
data_2b = pd.DataFrame(d)
data_2b.columns = ['subject_id','hits_RL','misses_RL','fa_RL','cr_RL','hits_LR','misses_LR','fa_LR','cr_LR','hits','misses','fa','cr']
data_2b.subject_id = data.subject_id.astype('str')
data_2b.subject_id = subjects[...]

In [12]:
for i, s in enumerate(subjects):
    print "Subject %d of %d" % (i+1, len(subjects))
    try:
        data1 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_RL/LINKED_DATA/EPRIME/%s_3T_WM_run1_TAB.txt' % (datapath, s, s))
        data2 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_LR/LINKED_DATA/EPRIME/%s_3T_WM_run2_TAB.txt' % (datapath, s, s))

        data_2b.hits_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] == data1['Stim.CRESP']),
                             data1['BlockType'] == '2-Back'))
        data_2b.misses_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] != data1['Stim.CRESP']),
                               data1['BlockType'] == '2-Back'))
        data_2b.fa_RL[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data1['TargetType'] == 'lure', data1['TargetType'] == 'nontarget'),
                data1['Stim.RESP'] != data1['Stim.CRESP']), data1['BlockType'] == '2-Back'))
        data_2b.cr_RL[i] = 40-data.fa_RL[i]-data.hits_RL[i]-data.misses_RL[i]

        data_2b.hits_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] == data2['Stim.CRESP']),
                             data2['BlockType'] == '2-Back'))
        data_2b.misses_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] != data2['Stim.CRESP']),
                               data2['BlockType'] == '2-Back'))
        data_2b.fa_LR[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data2['TargetType'] == 'lure', data2['TargetType'] == 'nontarget'),
                data2['Stim.RESP'] != data2['Stim.CRESP']), data2['BlockType'] == '2-Back'))
        data_2b.cr_LR[i] = 40-data.fa_LR[i]-data.hits_LR[i]-data.misses_LR[i]
    except:
        print "Data for subject %s does not exist" % s

data_2b.hits = data.hits_RL + data.hits_LR
data_2b.misses = data.misses_RL + data.misses_LR
data_2b.fa = data.fa_RL + data.fa_LR
data_2b.cr = data.cr_RL + data.cr_LR

,0,ACC_0b_run1,ACC_0b_run2,ACC_2b_run1,ACC_2b_run2,ACC_0b,ACC_2b
0,100307,0.750,1.000,0.625,0.625,0.8750,0.6250
1,100408,0.750,0.875,1.000,1.000,0.8125,1.0000
2,101006,0.750,0.750,0.125,0.125,0.7500,0.1250
3,101107,0.875,0.875,0.500,0.750,0.8750,0.6250
4,101309,0.625,1.000,0.875,1.000,0.8125,0.9375
5,101410,0.750,0.750,0.875,1.000,0.7500,0.9375
6,101915,0.750,1.000,0.750,0.625,0.8750,0.6875
7,102008,0.875,1.000,1.000,0.875,0.9375,0.9375
8,102311,0.750,0.250,0.500,0.875,0.5000,0.6875
9,102816,1.000,1.000,0.625,0.875,1.0000,0.7500


In [ ]:
data_2b.to_csv('../data/WM_2-back_performance.csv')

### 0-back

In [ ]:
d = np.zeros([494, 13])
data_0b = pd.DataFrame(d)
data_0b.columns = ['subject_id','hits_RL','misses_RL','fa_RL','cr_RL','hits_LR','misses_LR','fa_LR','cr_LR','hits','misses','fa','cr']
data_0b.subject_id = data.subject_id.astype('str')
data_0b.subject_id = subjects[...]
data_0b.iloc[:,1:] = np.nan

In [ ]:
for i, s in enumerate(subjects):
    print "Subject %d of %d" % (i+1, len(subjects))
    try:
        data1 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_RL/LINKED_DATA/EPRIME/%s_3T_WM_run1_TAB.txt' % (datapath, s, s))
        data2 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_LR/LINKED_DATA/EPRIME/%s_3T_WM_run2_TAB.txt' % (datapath, s, s))

        data_0b.hits_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] == data1['Stim.CRESP']),
                             data1['BlockType'] == '0-Back'))
        data_0b.misses_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] != data1['Stim.CRESP']),
                               data1['BlockType'] == '0-Back'))
        data_0b.fa_RL[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data1['TargetType'] == 'lure', data1['TargetType'] == 'nontarget'),
                data1['Stim.RESP'] != data1['Stim.CRESP']), data1['BlockType'] == '0-Back'))
        data_0b.cr_RL[i] = 40-data.fa_RL[i]-data.hits_RL[i]-data.misses_RL[i]

        data_0b.hits_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] == data2['Stim.CRESP']),
                             data2['BlockType'] == '0-Back'))
        data_0b.misses_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] != data2['Stim.CRESP']),
                               data2['BlockType'] == '0-Back'))
        data_0b.fa_LR[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data2['TargetType'] == 'lure', data2['TargetType'] == 'nontarget'),
                data2['Stim.RESP'] != data2['Stim.CRESP']), data2['BlockType'] == '0-Back'))
        data.cr_LR[i] = 40-data.fa_LR[i]-data.hits_LR[i]-data.misses_LR[i]
    except:
        print "Data for subject %s does not exist" % s

data_0b.hits = data.hits_RL + data.hits_LR
data_0b.misses = data.misses_RL + data.misses_LR
data_0b.fa = data.fa_RL + data.fa_LR
data_0b.cr = data.cr_RL + data.cr_LR

In [ ]:
data_0b.to_csv('../data/WM_0-back_performance.csv')

## Use the extracted information to calculate d' and c'

The SDT parameters are estimated for each condition separately. We apply a hierarchical bayesian model described in Lee & Wagenmakers (2014).

Lee, M.D. and Wagenmakers, E.J., 2014. Bayesian cognitive modeling: A practical course. Cambridge University Press.

In [45]:
data_0b = pd.read_csv('../data/WM_0-back_performance.csv', index_col = 0)
data_2b = pd.read_csv('../data/WM_2-back_performance.csv', index_col = 0)

data_0b = data_0b.dropna()
data_2b = data_2b.dropna()

data_0b.iloc[:,1:] = data_0b.iloc[:,1:].astype(int)
data_2b.iloc[:,1:] = data_2b.iloc[:,1:].astype(int)

In [47]:
SDT_model = '../fgrad/SDT_single.stan'

### Sampling for each run separately

In [48]:
stan_2b_RL_data = {'h': data_2b['hits_RL'],
                  'f' : data_2b['fa_RL'],
                  's' : data_2b['hits_RL'] + data_2b['misses_RL'],
                  'n' : data_2b['fa_RL'] + data_2b['cr_RL'],
                  'k' : data_2b.shape[0]}

sdt_2b_RL = pystan.stan(file=SDT_model, data=stan_2b_RL_data, iter=10000, chains=4)

In [50]:
stan_2b_LR_data = {'h': data_2b['hits_LR'],
                  'f' : data_2b['fa_LR'],
                  's' : data_2b['hits_LR'] + data_2b['misses_LR'],
                  'n' : data_2b['fa_LR'] + data_2b['cr_LR'],
                  'k' : data_2b.shape[0]}

sdt_2b_LR = pystan.stan(file=SDT_model, data=stan_2b_LR_data, iter=10000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8079f2c55ca4cc5bc0e2862533a84d47 NOW.


In [51]:
stan_0b_RL_data = {'h': data_0b['hits_RL'],
                  'f' : data_0b['fa_RL'],
                  's' : data_0b['hits_RL'] + data_0b['misses_RL'],
                  'n' : data_0b['fa_RL'] + data_0b['cr_RL'],
                  'k' : data_0b.shape[0]}

sdt_0b_RL = pystan.stan(file=SDT_model, data=stan_0b_RL_data, iter=10000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8079f2c55ca4cc5bc0e2862533a84d47 NOW.


In [52]:
stan_0b_LR_data = {'h': data_0b['hits_LR'],
                  'f' : data_0b['fa_LR'],
                  's' : data_0b['hits_LR'] + data_0b['misses_LR'],
                  'n' : data_0b['fa_LR'] + data_0b['cr_LR'],
                  'k' : data_0b.shape[0]}

sdt_0b_LR = pystan.stan(file=SDT_model, data=stan_0b_LR_data, iter=10000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8079f2c55ca4cc5bc0e2862533a84d47 NOW.


### Assemble the means into a single data table

In [79]:
SDT_nback = pd.DataFrame(data_2b['subject_id'])
SDT_nback['dprime_2b_RL'] = np.nan
SDT_nback['dprime_2b_LR'] = np.nan
SDT_nback['dprime_0b_RL'] = np.nan
SDT_nback['dprime_0b_LR'] = np.nan
SDT_nback['bias_2b_RL'] = np.nan
SDT_nback['bias_2b_LR'] = np.nan
SDT_nback['bias_0b_RL'] = np.nan
SDT_nback['bias_0b_LR'] = np.nan

In [80]:
sdt_2b_RL_params = sdt_2b_RL.extract(permuted = False, inc_warmup=False)
sdt_2b_RL_means = np.mean(np.mean(sdt_2b_RL_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_2b_RL'] = sdt_2b_RL_means[460:]
SDT_nback['bias_2b_RL'] = sdt_2b_RL_means[0:460]

In [82]:
sdt_2b_LR_params = sdt_2b_LR.extract(permuted = False, inc_warmup=False)
sdt_2b_LR_means = np.mean(np.mean(sdt_2b_LR_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_2b_LR'] = sdt_2b_LR_means[460:]
SDT_nback['bias_2b_LR'] = sdt_2b_LR_means[0:460]

In [83]:
sdt_0b_RL_params = sdt_0b_RL.extract(permuted = False, inc_warmup=False)
sdt_0b_RL_means = np.mean(np.mean(sdt_0b_RL_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_0b_RL'] = sdt_0b_RL_means[460:]
SDT_nback['bias_0b_RL'] = sdt_0b_RL_means[0:460]

In [84]:
sdt_0b_LR_params = sdt_0b_LR.extract(permuted = False, inc_warmup=False)
sdt_0b_LR_means = np.mean(np.mean(sdt_0b_LR_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_0b_LR'] = sdt_0b_LR_means[460:]
SDT_nback['bias_0b_LR'] = sdt_0b_LR_means[0:460]

In [86]:
SDT_nback.to_csv('../data/WM_SDT.csv')